# Creating fibers using 3D coordinates

```{note}
This tutorial will use a lot of code explained in the [simulation tutorial](2_sim_and_activation.ipynb) and the [resampling potentials tutorial](4_resampling_potentials.ipynb), so it is recommended to review them before proceeding.
```

## Creating 3D fiber path and potentials
Normally, you might use 3D fiber coordinates alongside potentials that you obtained from an external source (e.g., COMSOL, ANSYS, etc.). For the purpose of this tutorial, we will create a fiber path using a 3D spiral and create potentials using a Gaussian curve. By default, the start of the fiber will be placed at the first coordinate along the path and extend until no more sections can be added. Later in this tutorial, we demonstrate how to shift the fiber to a specific location (typically to achieve some positioning of nodes of Ranvier for myelinated fibers).

In [ ]:
# Creating the spiral path for the fiber
import numpy as np
import matplotlib.pyplot as plt

# Define parameters for the spiral
height = 1000  # total height of the spiral
turns = 5  # number of turns in the spiral
radius = 1000  # radius of the spiral
points_per_turn = 20  # points per turn

# Generate the spiral coordinates
t = np.linspace(0, 2 * np.pi * turns, points_per_turn * turns)
x = radius * np.cos(t)
y = radius * np.sin(t)
z = np.linspace(0, height, points_per_turn * turns)

# Combine into a single array for path coordinates
path_coordinates = np.column_stack((x, y, z))

# Calculate the length along the fiber
fiber_length = np.sqrt(np.sum(np.diff(path_coordinates, axis=0) ** 2, axis=1)).sum()

# Generate the Gaussian curve of potentials
# Set the mean at the center of the fiber and standard deviation as a fraction of the fiber length
mean = fiber_length / 2
std_dev = fiber_length / 50  # Smaller value gives a narrower peak

# Create an array representing the linear distance along the fiber
linear_distance = np.linspace(0, fiber_length, len(path_coordinates))

# Generate Gaussian distributed potentials
potentials = np.exp(-((linear_distance - mean) ** 2 / (2 * std_dev**2)))

# Normalize potentials for better visualization or use
potentials /= np.max(potentials)

# Scale to a maximum of 500 mV
potentials *= 500

# Create subplots for both the 3D path and potentials
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Plot the generated path to visualize it
ax1 = fig.add_subplot(121, projection='3d')
ax1.plot(x, y, z)
ax1.set_title("3D Spiral Path for Fiber")
ax1.set_xlabel("X axis")
ax1.set_ylabel("Y axis")
ax1.set_zlabel("Z axis")

# Plot the Gaussian distribution of potentials along the fiber
ax2.plot(linear_distance, potentials, label='Gaussian Potentials')
ax2.set_xlabel('Distance along fiber (µm)')
ax2.set_ylabel('Potential (mV)')
ax2.set_title('Gaussian Distribution of Potentials along fiber path')
ax2.legend()

plt.tight_layout()
plt.show()

## Resampling 3D potentials
For this tutorial, we will create a fiber model using the MRG model using the same process as the [fiber creation tutorial](1_create_fiber.ipynb). Instead of specifying the length or number of sections, we provide the coordinates of the 3D fiber path. Since we want a 3D fiber, we must use the function {py:func}`~pyfibers.fiber.build_fiber_3d()` instead.

In [ ]:
from pyfibers import FiberModel, build_fiber_3d

fiber = build_fiber_3d(
    FiberModel.MRG_INTERPOLATION, diameter=10, path_coordinates=path_coordinates
)
print(fiber)
print('Fiber is 3D?', fiber.is_3d())

We can either:
- use the {py:meth}`~pyfibers.fiber.Fiber.resample_potentials_3d()` method of the fiber object to get the potentials at the center of each fiber section. (not recommended)
- or calculate arc lengths and use {py:meth}`~pyfibers.fiber.Fiber.resample_potentials()` to get the potentials. (recommended)

In [ ]:
# Using the resample_potentials_3d() method to resample the potentials along the fiber path
fiber.resample_potentials_3d(
    potentials=potentials, potential_coords=path_coordinates, center=True, inplace=True
)

plt.figure()
plt.plot(
    fiber.longitudinal_coordinates,
    fiber.potentials,
    marker='o',
    label='resample_potentials_3d()',
)
plt.xlabel('Distance along fiber (µm)')
plt.ylabel('Potential (mV)')
plt.title('Resampled Potentials along Fiber')

# Calculating arc lengths and using resample_potentials()
arc_lengths = np.concatenate(
    ([0], np.cumsum(np.sqrt(np.sum(np.diff(path_coordinates, axis=0) ** 2, axis=1))))
)
fiber.resample_potentials(
    potentials=potentials, potential_coords=arc_lengths, center=True, inplace=True
)
plt.plot(
    fiber.longitudinal_coordinates,
    fiber.potentials,
    marker='x',
    label='resample_potentials()',
    alpha=0.6,
    color='k',
)

## Simulation
We will use the same simulation setup as before.

In [ ]:
import numpy as np
from pyfibers import ScaledStim
from scipy.interpolate import interp1d

# Setup for simulation
time_step = 0.001  # milliseconds
time_stop = 15  # milliseconds
start, on, off = 0, 0.1, 0.2
waveform = interp1d(
    [start, on, off, time_stop], [1, -1, 0, 0], kind="previous"
)  # biphasic rectangular pulse

# Create stimulation object
stimulation = ScaledStim(waveform=waveform, dt=time_step, tstop=time_stop)

and then run a fixed amplitude simulation:

In [ ]:
ap, time = stimulation.run_sim(-1.5, fiber)
print(f'Number of action potentials detected: {ap}')
print(f'Time of last action potential detection: {time} ms')

## Creating potentials from a point source
In the previous example, we created potentials using a Gaussian curve. In this example, we will create potentials using a point source. We will use the same fiber path as before, but we will calculate the potentials from a point source near the 3D fiber path.

In [ ]:
# Calculate point source potentials at all fiber coordinates
x, y, z = 800, 800, 500  # Point source location
i0 = 1  # Current of the point source
sigma = 1  # Isotropic conductivity
fiber_potentials = fiber.point_source_potentials(x, y, z, i0, sigma, inplace=True)

# Create subplots for both the potentials and 3D visualization
fig = plt.figure(figsize=(15, 6))

# Plot potentials
ax1 = plt.subplot(121)
ax1.plot(fiber.longitudinal_coordinates, fiber.potentials, marker='o', markersize=4)
ax1.set_xlabel('Distance along fiber (µm)')
ax1.set_ylabel('Potential (mV)')
ax1.set_title('Fiber potentials from point source')
ax1.grid(True, alpha=0.3)

# Plot the fiber with the point source
ax2 = plt.subplot(122, projection='3d')
ax2.plot(x, y, z, 'ro', label='Point Source', markersize=8)
ax2.plot(
    fiber.coordinates[:, 0],
    fiber.coordinates[:, 1],
    fiber.coordinates[:, 2],
    label='Fiber Path',
    linewidth=2,
)
ax2.set_title('Fiber Path with Point Source')
ax2.set_xlabel('X axis')
ax2.set_ylabel('Y axis')
ax2.set_zlabel('Z axis')
ax2.legend()

plt.tight_layout()
plt.show()

The potentials have several peaks, which would be expected from the location where we placed the point source.

## Creating fibers with a shift along the 3D path
The prior examples created fibers with the start of the fiber at the beginning of the path. Users may wish to specify a shift along this 3D path to achieve a specific positioning of nodes of Ranvier for myelinated fibers. We can achieve this by specifying the shift along the 3D path using the `shift` parameter in the {py:func}`~pyfibers.fiber.build_fiber_3d()` function. For this example, we will use a path in 2D space to make it easier to visualize the shift.

We can see that the fiber has been created starting at the beginning of the path. We might want to center the fiber along the path. We can do this by setting `center=True`.

Note how for one shift example (centered + shifting by half internodal length) it was only possible to fit 14 nodes (vs. 15 in every other plot). By default, fibers will not be created with even node counts, and so an additional node was removed to bring the count to 13.

In [ ]:
# Generate a right angle for fiber path
x = [0, 10000, 10000]
y = [0, 0, 10000]
z = [0, 0, 0]
path_coordinates = np.array([x, y, z]).T

model = FiberModel.MRG_INTERPOLATION


def fiber_plot(ax, fiber, title):
    """Plot the fiber path and coordinates on the given axis."""  # noqa: DAR101
    # Plot fiber path
    ax.plot(x, y, lw=10, color='gray', alpha=0.3, label='fiber path')
    # Plot fiber coordinates
    ax.plot(
        fiber.coordinates[:, 0],
        fiber.coordinates[:, 1],
        marker='o',
        markersize=3,
        lw=0,
        label='fiber coordinates',
        color='black',
    )
    ax.plot(
        fiber.coordinates[:, 0][::11],
        fiber.coordinates[:, 1][::11],
        marker='o',
        markersize=3,
        lw=0,
        c='r',
        label='nodes',
    )
    ax.set_title(title)
    ax.set_aspect('equal')
    ax.legend()


# Define fiber configurations as a list of parameter dictionaries
fiber_configs = [
    {'title': 'No shift', 'center': False, 'shift': None, 'shift_ratio': None},
    {'title': 'Shifted by 200 µm', 'center': False, 'shift': 200, 'shift_ratio': None},
    {
        'title': 'Shifted by half internodal length',
        'center': False,
        'shift': None,
        'shift_ratio': 0.5,
    },
    {'title': 'Centered', 'center': True, 'shift': None, 'shift_ratio': None},
    {
        'title': 'Centered + shifted by 200 µm',
        'center': True,
        'shift': 200,
        'shift_ratio': None,
    },
    {
        'title': 'Centered + shifted by half internodal length',
        'center': True,
        'shift': None,
        'shift_ratio': 0.5,
    },
]

# Create fibers with different shift configurations
fibers = {}
titles = {}

for config in fiber_configs:
    # Build fiber with the specified parameters
    fiber_params = {
        'diameter': 13,
        'fiber_model': model,
        'temperature': 37,
        'path_coordinates': path_coordinates,
        'passive_end_nodes': 2,
    }

    # Add optional parameters if specified
    if config['center']:
        fiber_params['center'] = True
    if config['shift'] is not None:
        fiber_params['shift'] = config['shift']
    if config['shift_ratio'] is not None:
        fiber_params['shift_ratio'] = config['shift_ratio']

    # Create fiber
    fiber = build_fiber_3d(**fiber_params)

    # Store fiber and title
    key = config['title'].lower().replace(' ', '_').replace('+', '').replace('µm', 'um')
    fibers[key] = fiber
    titles[key] = config['title']

# Create subplots for all fiber configurations
fig, axes = plt.subplots(2, 3, figsize=(12, 8), sharex=True, sharey=True)
axes = axes.flatten()

for i, (key, fiber) in enumerate(fibers.items()):
    fiber_plot(axes[i], fiber, titles[key])

plt.tight_layout()
plt.show()

We might want to specify a `shift` amount along the path. We can do this by setting `shift` to a value in micrometers, or `shift_ratio` to a float that represents a proportion of the fiber's internodal length. Note that distances greater than one internodal length will only be shifted by the remainder (e.g., for a 16 µm diameter fiber with an internodal length of 1600 µm, a `shift` of 2100 µm will be equivalent to a `shift` of 500 µm, and a `shift_ratio` of 1.5 is equivalent to a `shift_ratio` of 0.5).